# 2.3 - Macrobond web API - Screening with a Keyword

*Performing coverage checks based on a keyword*

This notebook aims to provide examples of how to use Macrobond's web API call methods as well as insights on the key attributes used to display the output in an understandable format.

We will focus here on using the Search method based on a **keyword** filter. This helps you build a list of time series potentially relating to a common theme.

*The examples uses the Web API, but you can chose to use the desktop COM API and get the same result. Full error handling is omitted for brevity*

***

## Importing packages

In [ ]:
import pandas
from matplotlib import pyplot

from macrobond_financial.web import WebClient

***

## Get the data - Keyword: Government Securities Auctions

Note that we are using here the Search endpoint with filters on keyword `text=Government Securities Auctions` and `Frequency=daily` time series in this example.

Feel free to refer to https://api.macrobondfinancial.com/swagger/index.html to get the comprehensive list of web API endpoints and parameters used.

***

## Visualising the data
Let's evaluate Macrobond's coverage for daily time series related to government securities auctions.

In [ ]:
with WebClient() as api:
    data_frame = api.entity_search(
        text="Government Securities Auctions",
        entity_types="TimeSeries",
        must_have_values={"Frequency": "daily"},
        include_discontinued=False,
    ).to_pd_data_frame()[
        [
            "Name",
            "FullDescription",
            "Description",
            "Region",
            "Category",
            "Frequency",
            "Source",
            "Class",
            "PriceType",
        ]
    ]
data_frame.head(10)

### Let's make sense of the attribute PriceType

We will use here the metadata endpoint from the web API as detailed in the notebook **1.1 - Macrobond web API - Metadata Navigation**

In [ ]:
with WebClient() as api:
    data_frame = api.metadata_get_attribute_information("PriceType")[0].to_pd_data_frame()
data_frame

As the PriceType uses values from list and is not a free text attribute, let's have a look at its values to be used later as a group.

In [ ]:
with WebClient() as api:
    df3 = api.metadata_list_values("PriceType").to_pd_data_frame()[
        ["value", "description", "comment"]
    ]
df3

Now that we get a sense of potential groupings we can achieve thanks to the metadata, let's manipulate the ones that will be used.

In [ ]:
with WebClient() as api:
    data_frame = api.entity_search(
        text="Government Securities Auctions",
        entity_types="TimeSeries",
        must_have_values={"Frequency": "daily"},
        include_discontinued=False,
    ).to_pd_data_frame()[
        [
            "Name",
            "FullDescription",
            "Description",
            "Region",
            "Category",
            "Frequency",
            "Source",
            "Class",
            "PriceType",
        ]
    ]

data_frame["RegionString"] = data_frame["Region"].apply(
    lambda x: "" if pandas.isna(x) else ", ".join(map(str, x))
)

# Need to account for NaN values before converting to a string
data_frame["PriceTypeS"] = data_frame["PriceType"].fillna("")
data_frame["PriceTypeString"] = data_frame["PriceTypeS"].apply(lambda x: ", ".join(map(str, x)))

# Mapping the PriceType to its descriptions
mapping = dict(df3[["value", "description"]].values)
data_frame["PriceTypeD"] = data_frame.PriceTypeString.map(mapping)
data_frame.head(1)

### Displaying the new DataFrame
Let's see how our transformations have been applied by isolating on a few columns: `df.iloc[rows,[columns]]`.

In [ ]:
df_final = data_frame.iloc[0:12000, [0, 1, 2, 3, 9, 4, 6, 12]]
df_final.head(10)

### Group the results by PriceType

In [ ]:
df_group1 = df_final.groupby(["PriceTypeD"])["Name"].count().reset_index(name="Count")
df_group1.sort_values(by="Count", ascending=False)

Let's isolate the top 10 results of our dataframe to keep the top price types.

In [ ]:
df_group1_top = df_group1.nlargest(10, "Count")

### Group the results by Region

In [ ]:
with WebClient() as api:
    df4 = api.metadata_list_values("Region").to_pd_data_frame()[["value", "description"]]

    # Mapping the Region value to its description
    mapping = dict(df4[["value", "description"]].values)
    df_final["RegionD"] = data_frame.RegionString.map(mapping)

    # Applying the grouping
    df_group2 = df_final.groupby(["RegionD"])["Name"].count().reset_index(name="Count")
    df_group2.sort_values(by="Count", ascending=False)
df_group2

In [ ]:
df_group2 = df_final.groupby(["RegionD", "PriceTypeD"])["Name"].count().reset_index(name="Count")
# df_group2.sort_values(['RegionD','Count'], ascending=False)
heatmap = df_group2.loc[
    (df_group2["RegionD"] == "Poland")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
    | (df_group2["RegionD"] == "United States")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
    | (df_group2["RegionD"] == "Sweden")
    & (
        (df_group2["PriceTypeD"] == "Yield")
        | (df_group2["PriceTypeD"] == "Offered")
        | (df_group2["PriceTypeD"] == "Price")
        | (df_group2["PriceTypeD"] == "Sales")
    )
]
heatmap

### Plot the Government Securities Auctions Top by type and region

In [ ]:
pyplot.rcParams["figure.figsize"] = [16, 9]

colours = [
    (27 / 255, 54 / 255, 93 / 255),
    (37 / 255, 107 / 255, 162 / 255),
    (142 / 255, 81 / 255, 168 / 255),
    (88 / 255, 162 / 255, 145 / 255),
    (205 / 255, 84 / 255, 91 / 255),
    (0 / 255, 79 / 255, 89 / 255),
    (246 / 255, 141 / 255, 46 / 255),
    (0 / 255, 133 / 255, 120 / 255),
    (147 / 255, 64 / 255, 84 / 255),
]
pyplot.bar(df_group1_top["PriceTypeD"], df_group1_top["Count"], color=colours)
pyplot.title("Pricing Type of Government Securities Auctions", fontsize=14)
pyplot.xlabel("PriceType", fontsize=14)
pyplot.ylabel("Count", fontsize=14)
pyplot.grid(False)
pyplot.autoscale()
pyplot.show()

Let's visualise the Price Type for Poland and Sweden only.

In [ ]:
df_final_filtered = df_final.loc[
    (df_final["RegionD"] == "Poland") | (df_final["RegionD"] == "Sweden")
]

In [ ]:
colours = [
    (27 / 255, 54 / 255, 93 / 255),
    (37 / 255, 107 / 255, 162 / 255),
    (142 / 255, 81 / 255, 168 / 255),
    (88 / 255, 162 / 255, 145 / 255),
    (205 / 255, 84 / 255, 91 / 255),
    (0 / 255, 79 / 255, 89 / 255),
    (246 / 255, 141 / 255, 46 / 255),
    (0 / 255, 133 / 255, 120 / 255),
    (147 / 255, 64 / 255, 84 / 255),
    (37 / 255, 40 / 255, 42 / 255),
    (51 / 255, 63 / 255, 72 / 255),
    (103 / 255, 109 / 255, 114 / 255),
    (199 / 255, 201 / 255, 199 / 255),
    (239 / 255, 240 / 255, 241 / 255),
]
df_final_filtered.groupby(["RegionD", "PriceTypeD"]).size().unstack().plot(
    kind="bar", stacked=True, color=colours
)
pyplot.show()